# Задание 3
Реализуйте регуляризованый LDA с двумя параметрами из лекции. Сравните его с логистической регрессией (осуществите тюнинг гиперпараметров для обеих моделей) на публичных датасетах (хотя бы 5) из UCI


In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris, load_wine, load_breast_cancer, make_classification
from scipy.linalg import eigh
from sklearn.base import BaseEstimator, ClassifierMixin
from itertools import product

In [2]:
class rLDA(BaseEstimator, ClassifierMixin):
    def __init__(self, alpha=0.5, gamma=0.5, epsilon=1e-6):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon  # докинем маленький эпсилон к значениям, чтобы было стабильней

    # ф-ция fit для обучения
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        n_features = X.shape[1]

        # подсчет медиан
        class_means = []
        for c in self.classes_:
            class_means.append(X[y == c].mean(axis=0))
        global_mean = X.mean(axis=0)

        cov_m_in = np.zeros((n_features, n_features)) # м-ца ковариации внутри класса, заполнили нулями
        for i, c in enumerate(self.classes_): # считаем ковариацию для каждого класса
            X_c = X[y == c] # смотрим на все эл-ты класса
            cov_m_in += np.cov(X_c, rowvar=False) * (len(X_c) - 1)

        cov_m_between = np.zeros((n_features, n_features)) # м-ца ковариации между классами, тоже сначала заполним нулями
        for i, c in enumerate(self.classes_):
            n_c = (y == c).sum() # число эл-тов в классе
            mean_diff = class_means[i] - global_mean
            cov_m_between += n_c * np.outer(mean_diff, mean_diff)

        # регуляризованная матрица ковариации по формулам из книги
        I = np.eye(n_features)
        S = self.alpha * cov_m_in + self.gamma * cov_m_between + (1 - self.alpha - self.gamma) * I
        S += self.epsilon * np.eye(n_features)

        # решаем задачу на собственные значения
        evals, evecs = eigh(cov_m_between, S)
        self.scalings_ = evecs[:, ::-1][:, :n_classes - 1] # собственные векторы, которые соответствуют наиб собственным значениям
        self.class_means_ = np.array(class_means)
        self.global_mean_ = global_mean

        return self

    def predict(self, X):
        X_projected = (X - self.global_mean_).dot(self.scalings_)
        projected_means = (self.class_means_ - self.global_mean_).dot(self.scalings_)
        distances = np.linalg.norm(X_projected[:, None, :] - projected_means[None, :, :], axis=2)
        return self.classes_[np.argmin(distances, axis=1)]

In [3]:
def best_alpha_gamma(X_train, y_train, X_test, y_test, alphas, gammas): # для поиска наилучших альфа и гамма
    best_alpha, best_gamma, best_score = None, None, -np.inf
    for alpha, gamma in product(alphas, gammas):
        if alpha + gamma <= 1:
            model = rLDA(alpha=alpha, gamma=gamma)
            model.fit(X_train, y_train)
            preds = model.predict(X_test)
            score = accuracy_score(y_test, preds)
            if score > best_score:
                best_alpha, best_gamma, best_score = alpha, gamma, score
    return best_alpha, best_gamma, best_score

Буду использовать датасеты:
1) вино - загружаем с помощью встроенной функции в sklearn.datasets load_wine()
2) iris, также загружаю встроенной функцией в sklearn.datasets load_iris()
3) breast cancer, тоже встроенной функцией load_breast_cancer()
4) ecoli
5) mushroom
6) glass

In [11]:
def load_mushroom():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"
    df = pd.read_csv(url, header=None)
    df = df.apply(LabelEncoder().fit_transform)
    X = df.iloc[:, 1:].values
    y = df.iloc[:, 0].values  # 0: съедобный, 1: ядовитый
    return X, y

def load_ecoli():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/ecoli/ecoli.data"
    df = pd.read_csv(url, delim_whitespace=True, header=None)
    X = df.iloc[:, 1:-1].values
    y = LabelEncoder().fit_transform(df.iloc[:, -1].values)
    return X, y

def load_glass():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data"
    column_names = [
        'id', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'type'
    ]
    df = pd.read_csv(url, header=None, names=column_names)
    df.drop(columns=['id'], inplace=True)
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values  # тип стекла
    return X, y

In [12]:
datasets = {
    "Iris": load_iris(as_frame=True),
    "Wine": load_wine(as_frame=True),
    "Breast Cancer": load_breast_cancer(as_frame=True),
    "Ecoli": load_ecoli(),
    "Mushroom": load_mushroom(),
    "Glass": load_glass()
}

In [13]:
scaler = StandardScaler()
alphas = np.linspace(0, 1, 10)
gammas = np.linspace(0, 1, 10)

In [14]:
results = [] # для хранения результатов
for dataset_name, data in datasets.items():
    if isinstance(data, tuple):
        X, y = data
        X = pd.DataFrame(X)
        y = pd.Series(y)
    else:
        X = data.data
        y = data.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # подбираем лучшие параметры для rLDA
    best_alpha, best_gamma, lda_score = best_alpha_gamma(X_train, y_train, X_test, y_test, alphas, gammas)

    # теперь посмотрим на логрег
    logistic = LogisticRegression(max_iter=500)
    logistic_params = {"C": np.logspace(-4, 4, 10)}
    logistic_search = GridSearchCV(logistic, param_grid=logistic_params, cv=5, scoring="accuracy")
    logistic_search.fit(X_train, y_train)
    logistic_best = logistic_search.best_estimator_
    logistic_score = accuracy_score(y_test, logistic_best.predict(X_test))

    results.append({
        "Dataset": dataset_name,
        "Best alpha": best_alpha,
        "Best gamma": best_gamma,
        "rLDA": lda_score,
        "LogReg": logistic_score
    })

/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
res_df = pd.DataFrame(results)
print(res_df)

         Dataset  Best alpha  Best gamma      rLDA    LogReg
0           Iris    0.111111    0.000000  1.000000  1.000000
1           Wine    0.000000    0.111111  1.000000  1.000000
2  Breast Cancer    0.111111    0.000000  0.973684  0.973684
3          Ecoli    0.000000    0.000000  0.808824  0.882353
4       Mushroom    0.111111    0.000000  0.945231  0.966154
5          Glass    0.111111    0.000000  0.744186  0.720930


Дополнительно: постарайтесь найти датасеты, где LDA работает лучше и попробуйте объяснить почему.
Подсказка: если данных достаточно много, то LDA и LogReg не будут отличаться, даже если предположения LDA выполняются, попробуйте уменьшать обучающую выборку, чтобы найти ситуации, когда LDA работает лучше. Если не получается с публичными датасетами - сгенерируйте (в этом случае посчитайте bias и variance для LDA и LogReg).

Из результатов видно, что результаты у rLDA и LogReg примерно одинаковы, однако, на датасете Ecoli rLDA дает результат хуже. Попробуем использовать подсказку и уменьшить обучающую выборку для этого датасета, посмотрим, что произойдет.

In [42]:
def change_train(X, y, train_sizes, alphas, gammas):
    results = []
    for train_size in train_sizes:
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=42)
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        best_alpha, best_gamma, lda_score = best_alpha_gamma(X_train, y_train, X_test, y_test, alphas, gammas)

        lr = LogisticRegression(max_iter=500)
        lr_params = {"C": np.logspace(-4, 4, 10)}
        lr_search = GridSearchCV(lr, param_grid=lr_params, cv=5, scoring="accuracy")
        lr_search.fit(X_train, y_train)
        lr_best = lr_search.best_estimator_
        score = accuracy_score(y_test, lr_best.predict(X_test))

        results.append({
            "Train Size": train_size,
            "Best alpha": best_alpha,
            "Best gamma": best_gamma,
            "rLDA": lda_score,
            "LogReg": score
        })
    return pd.DataFrame(results)
X_ecoli, y_ecoli = load_ecoli()
train_sizes = np.linspace(0.1, 0.8, 8)
ecoli_results = change_train(X_ecoli, y_ecoli, train_sizes, alphas, gammas)

print(ecoli_results)

/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/Users/elizabeth/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less

   Train Size  Best alpha  Best gamma      rLDA    LogReg
0         0.1    0.222222    0.111111  0.818482  0.818482
1         0.2    0.222222    0.222222  0.836431  0.814126
2         0.3    0.000000    0.000000  0.830508  0.822034
3         0.4    0.111111    0.000000  0.861386  0.816832
4         0.5    0.111111    0.000000  0.869048  0.869048
5         0.6    0.111111    0.000000  0.881481  0.896296
6         0.7    0.000000    0.000000  0.841584  0.881188
7         0.8    0.000000    0.000000  0.808824  0.882353


Видно, что подсказка сработала и результаты rLDA становятся лучше, когда уменьшаем размер train. До этого брали размер train = 0.8 от выборки, сейчас видим, что наилучший результат rLDA дает при train size = 0.6, а при train size = 0.4 результат rLDA лучше результата LogReg

Проверим, что в этом случае у нас выполнено предположение rLDA, то есть нормальность и равенство матриц ковариаций

In [87]:
from scipy.stats import shapiro
from numpy.linalg import LinAlgError

def norm(X_train, y_train):
    # Список классов
    classes = np.unique(y_train)

    # Проверка нормальности и ковариаций
    valid_classes = []  # Для классов с достаточным количеством данных
    class_covs = []
    normality_results = {}

    # Нормальность для каждого признака
    for c in classes:
        X_c = X_train[y_train == c]
        if X_c.shape[0] < 3:  # Пропускаем классы с недостаточным количеством данных
            continue

        try:
            cov_matrix = np.cov(X_c, rowvar=False)
            class_covs.append(cov_matrix)
            normality_results[f"Class {c}"] = [
                shapiro(X_c[:, i]).pvalue if X_c[:, i].size >= 3 else None for i in range(X_c.shape[1])
            ]
            valid_classes.append(c)  # Сохраняем только валидные классы
        except LinAlgError:
            print(f"Class {c}: Covariance matrix is singular and will be skipped.")


    # Вывод нормальности
    for class_name, p_values in normality_results.items():
        print(f"\n{class_name}:")
        for i, pval in enumerate(p_values):
            if pval is not None:
                print(f"Feature {i}: P-value={pval:.4f}")
            else:
                print(f"Feature {i}: Not enough data for test")

X_ecoli, y_ecoli = load_ecoli()
X_train, X_test, y_train, y_test = train_test_split(X_ecoli, y_ecoli, train_size=0.4, random_state=42)
norm(X_train, y_train)


Class 0:
Feature 0: P-value=0.6802
Feature 1: P-value=0.0554
Feature 2: P-value=1.0000
Feature 3: P-value=1.0000
Feature 4: P-value=0.8010
Feature 5: P-value=0.4561
Feature 6: P-value=0.0047

Class 1:
Feature 0: P-value=0.2865
Feature 1: P-value=0.1084
Feature 2: P-value=1.0000
Feature 3: P-value=1.0000
Feature 4: P-value=0.0010
Feature 5: P-value=0.1361
Feature 6: P-value=0.0003

Class 4:
Feature 0: P-value=0.6973
Feature 1: P-value=0.5696
Feature 2: P-value=0.0000
Feature 3: P-value=1.0000
Feature 4: P-value=0.0003
Feature 5: P-value=0.2897
Feature 6: P-value=0.0027

Class 5:
Feature 0: P-value=0.8811
Feature 1: P-value=0.9637
Feature 2: P-value=0.0000
Feature 3: P-value=1.0000
Feature 4: P-value=0.0995
Feature 5: P-value=0.4145
Feature 6: P-value=0.0203

Class 7:
Feature 0: P-value=0.0001
Feature 1: P-value=0.8066
Feature 2: P-value=1.0000
Feature 3: P-value=1.0000
Feature 4: P-value=0.3260
Feature 5: P-value=0.6275
Feature 6: P-value=0.6104


/Users/elizabeth/anaconda3/lib/python3.11/site-packages/scipy/stats/_morestats.py:1879: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


Видно, что большинство p-value имеют значение > 0.05, то есть условие нормальности выполнено

Теперь проверим равенство

In [89]:
import numpy as np
import pandas as pd
from scipy.stats import levene

# проверим равенство матриц ковариации
def equality(X_train, y_train):
    unique_classes = np.unique(y_train)
    covariances = []

    # считаем матрицы ковариации
    for cls in unique_classes:
        X_cls = X_train[y_train == cls]
        covariance_matrix = np.cov(X_cls, rowvar=False)
        covariances.append(covariance_matrix)

    # проверка равенства
    equal_cov_results = []
    for i in range(len(covariances)):
        for j in range(i + 1, len(covariances)):
            flattened_cov1 = covariances[i].flatten()
            flattened_cov2 = covariances[j].flatten()
            _, p_value_equal_cov = levene(flattened_cov1, flattened_cov2)
            equal_cov_results.append({
                "Classes": f"{unique_classes[i]}, {unique_classes[j]}",
                "P-Value": p_value_equal_cov
            })

    equal_cov_df = pd.DataFrame(equal_cov_results)
    return equal_cov_df

X_ecoli, y_ecoli = load_ecoli()
X_train, X_test, y_train, y_test = train_test_split(X_ecoli, y_ecoli, train_size=0.4, random_state=42)
equal_cov = equality(X_train, y_train)


print(equal_cov)

   Classes   P-Value
0     0, 1  0.114932
1     0, 4  0.005548
2     0, 5  0.209036
3     0, 6  0.544117
4     0, 7  0.893454
5     1, 4  0.781857
6     1, 5  0.400252
7     1, 6  0.625196
8     1, 7  0.126967
9     4, 5  0.099675
10    4, 6  0.369192
11    4, 7  0.006068
12    5, 6  0.446047
13    5, 7  0.238472
14    6, 7  0.491074


Видно, что с равенством матриц ковариации также все ок, то есть предположение rLDA выполнено, это доказывает нам хороший результат